In [1]:
import os
from pathlib import Path

# Document folder

In [2]:
doc_dir = "private-test-input"

files = list(Path(doc_dir).glob("*"))
print(files)

[WindowsPath('private-test-input/Public001.pdf'), WindowsPath('private-test-input/Public002.pdf'), WindowsPath('private-test-input/Public003.pdf'), WindowsPath('private-test-input/Public004.pdf'), WindowsPath('private-test-input/Public005.pdf'), WindowsPath('private-test-input/question.csv')]


In [3]:
file = files[0]
print(file)

private-test-input\Public001.pdf


In [4]:
import torch

torch.cuda.is_available()

True

# Basic usage

Using `DocumentConverter` to process a document.

In [5]:
from docling.document_converter import DocumentConverter

d:\Utilities\miniconda\envs\t\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
source = file
converter = DocumentConverter()
doc = converter.convert(source).document

2025-10-15 23:16:05,486 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-10-15 23:16:05,521 - INFO - Going to convert document batch...
2025-10-15 23:16:05,523 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 4f2edc0f7d9bb60b38ebfecf9a2609f5
2025-10-15 23:16:05,532 - INFO - Loading plugin 'docling_defaults'
2025-10-15 23:16:05,532 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-10-15 23:16:05,543 - INFO - Loading plugin 'docling_defaults'
2025-10-15 23:16:05,549 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-10-15 23:16:05,550 - INFO - rapidocr cannot be used because onnxruntime is not installed.
2025-10-15 23:16:05,551 - INFO - easyocr cannot be used because it is not installed.
2025-10-15 23:16:05,856 - INFO - Accelerator device: 'cuda:0'
[INFO] 2025-10-15 23:16:05,874 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-10-15 23:16:05,879 [RapidOCR] download_file.py:68: 

Serialize the converted result with `export_to_...` command

In [7]:
print(doc.export_to_markdown()[:500])

<!-- image -->

<!-- image -->

## VIETTEL AI RACE

## NGHIÊN C Ứ U IOT Ứ NG D Ụ NG XÂY D Ự NG MÔ HÌNH NHÀ THÔNG MINH

TD001

L ầ n ban hành: 1

Công ngh ệ ứ ng d ụ ng vào mô hình nhà thông minh d ự a trên Internet v ạ n v ậ t (IoT - Internet of Things) ph ầ n l ớ n b ị h ạ n ch ế và phân tán. Các đánh giá trong bài viết đượ c th ự c hi ện để phân lo ạ i b ố i c ả nh nghiên c ứ u v ề ứ ng d ụ ng IoT xây d ự ng mô hình nhà thông minh, nh ằ m cung c ấ p nh ữ ng hi ể u bi ế t có giá tr ị v ề công n


# Annotate images with OpenAI API call

In [8]:
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import (
    PdfPipelineOptions,
    PictureDescriptionApiOptions
)
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling_core.types.doc.document import ImageRefMode

def convert_with_image_annotation(input_doc_path):
    api_key = os.environ.get("OPENAI_API_KEY")
    model = "gpt-4.1-mini"
    picture_desc_api_option = PictureDescriptionApiOptions(
        url="https://api.openai.com/v1/chat/completions",
        prompt="Describe this image in sentences in a single paragraph.",
        params=dict(
                model=model,
            ),
        headers={
            "Authorization": "Bearer " + api_key,
        },
        timeout=60,
    )

    pipeline_options = PdfPipelineOptions(
        do_picture_description=True,
        picture_description_options=picture_desc_api_option,
        enable_remote_services=True,
        generate_picture_images=True,
        images_scale=2,
    )

    converter = DocumentConverter(
        format_options={InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)}
    )
    conv_res = converter.convert(source=input_doc_path)
    return conv_res

In [9]:
result = convert_with_image_annotation(file)

TypeError: can only concatenate str (not "NoneType") to str

In [ ]:
print(result.document.export_to_markdown(mark_annotations = True, include_annotations=True))

<!-- image -->

## Docling Technical Report

Version 1.0

Christoph Auer Maksym Lysak Ahmed Nassar Michele Dolfi Nikolaos Livathinos Panos Vagenas Cesar Berrospi Ramis Matteo Omenetti Fabian Lindlbauer Kasper Dinkla Lokesh Mishra Yusik Kim Shubham Gupta Rafael Teixeira de Lima Valery Weber Lucas Morin Ingmar Meijer Viktor Kuropiatnyk Peter W. J. Staar

AI4K Group, IBM Research R¨ uschlikon, Switzerland

## Abstract

This technical report introduces Docling , an easy to use, self-contained, MITlicensed open-source package for PDF document conversion. It is powered by state-of-the-art specialized AI models for layout analysis (DocLayNet) and table structure recognition (TableFormer), and runs efficiently on commodity hardware in a small resource budget. The code interface allows for easy extensibility and addition of new features and models.

## 1 Introduction

Converting PDF documents back into a machine-processable format has been a major challenge for decades due to their huge variabi

# Save to MD with external referenced images

In [ ]:
def export_function_md_with_image_ref(conv_res, output_path:str, replace_blank:str="_"):

    output_dir = Path(output_path)
    output_dir.mkdir(parents=True, exist_ok=True)
    doc_filename = conv_res.input.file.stem.replace(" ", replace_blank)

    # Save markdown with externally referenced pictures
    md_filename = output_dir / f"{doc_filename}-with-image-refs.md"
    conv_res.document.save_as_markdown(md_filename, image_mode=ImageRefMode.REFERENCED, include_annotations=True)

In [ ]:
export_function_md_with_image_ref(result, "outputs")